In [2]:
import pandas as pd
from pandas import DataFrame as DataFrame
from xlrd import open_workbook
import numpy as np
import re
import xlrd
def df_2_create_sql(db_name,tbl_name,df):
    create= "CREATE TABLE IF NOT EXISTS `{}`.`{}` \n(\n".format(db_name,tbl_name)
    for r in df.values:
        pk = ""
        increase = ""
        if pd.isnull(r[3])==False:
            if r[3].find("主键") !=-1:
                pk = "PRIMARY KEY"
            if r[3].find("自增长")!=-1:
                increase = "AUTO_INCREMENT"
        create += "    `{}` {}  {} {} COMMENT '{}',\n".format(r[0],r[2],pk,increase,r[1])
    create=create.rstrip(",\n")

    create +="\n);\n"
    return create


def excel_2_create_sql(db_name,excel_file,sheet_list=[]):
    if sheet_list==[]:
        sheet_list=open_workbook(excel_file).sheet_names()
    rettxt = ""
    for sheet in sheet_list:
        if sheet.find("cfg_")!=0 and sheet.find("tbl_")!=0:
            continue
        df = pd.read_excel(excel_file,sheet,usecols=[1,2,3,4],skiprows=[0,1,2,3])
        rettxt = rettxt + df_2_create_sql(db_name,sheet,df)
    return rettxt



# 转换字符串格式为驼峰式。
def change_2_big(str):
    list = str.split("_")
    for index in range(len(list)):
        list[index] = list[index].lower().capitalize()
    newstr = ''.join(list)
    return newstr

# 将接口数据转化为代码格式。
def service_2_model(data):
    urltxt = ""
    for list in data:
        newword = ""
        bigword = ""
        wordlist = list.split("/")
        if(wordlist[-1].find("(") == -1):
            newword = wordlist[-1]
            bigword = change_2_big(wordlist[-1])
        else:
            newword = wordlist[-2]+wordlist[-1]
            bigword = change_2_big(wordlist[-2])
        urltxt+="url(r'^"+newword+"$', views."+bigword+".as_view()),\n"
    return urltxt

# 将文件中的接口转化为数组。
def read_txt_2_list(txt_path,search_name):
    service_list = []
    line_list = open(txt_path, 'r',encoding='UTF-8').readlines()
    for line in line_list:
        service_line = ""
        if(line.find(search_name) != -1):
            service_line = line.split(":")[1].strip()
            service_list.append(service_line)
    back_data = service_2_model(service_list)
    return back_data

def df_2_model(db_name,tbl_name,df,sheet):
    cell_name = sheet.cell_value(0,1)
    pattern = re.compile(r".*\((.*?)\).*")


    new_tbl_name = ""
    ss = cell_name.split("_")
#     if len(ss)<=2:
    for s in ss:
        new_tbl_name +=s.capitalize()
#     else:
#         for s in ss[2:]:
#             new_tbl_name +=s.capitalize()
            
    model_str= "class {}(models.Model):\n".format(new_tbl_name)
    for r in df.values:
        try:
            if r[0].lower()=="id":
                continue
            type_str = ""
            if r[1].lower().strip()=="int":
                type_str = "models.IntegerField(null=True)"

            elif r[1].lower().strip()=="bigint":   
                type_str = "models.BigIntegerField(null=True)"

            elif r[1].lower().find("datetime")!=-1:   
                type_str = "models.DateTimeField(null=True)"

            elif r[1].lower().find("varchar")!=-1:
                varchar_len = int(r[2])       
                type_str = "models.CharField(max_length={},null=True)".format(varchar_len)

            elif r[1].lower().strip()=="float":   
                type_str = "models.FloatField(null=True)"            

            elif r[1].lower().find("smallint")!=-1 or r[2].lower().find("tinyint")!=-1:   
                type_str = "models.SmallIntegerField(null=True)"          

            elif r[1].lower().find("text")!=-1:
                type_str = "models.TextField(null=True)" 

            else:
                print("Error:{}".format(r))

            model_str += "    {} = {}\n".format(r[0].lower(),type_str)
            
        except Exception as e:
            print("Exception:{},{}\n{}".format(new_tbl_name,r,e))

    model_str += "\n    class Meta:\n        db_table = '{}'\n\n".format(cell_name)
    return model_str

def excel_2_model(db_name,excel_file,sheet_list=[]):
    newexcel = xlrd.open_workbook(excel_file)
    if sheet_list==[]:
        sheet_list=open_workbook(excel_file).sheet_names()
    rettxt = ""
    for sheetname in sheet_list:
        if sheetname.find("版本")==0 and sheetname.find("目录")==0:
            continue
        sheet = newexcel.sheet_by_name(sheetname)
        df = pd.read_excel(excel_file,sheetname,usecols=[0,1,2,3,4],skiprows=[0,1,2,3,4])
        rettxt = rettxt + df_2_model(db_name,sheetname,df,sheet)
    return rettxt
        

In [4]:
# 读取数据，并将文件中的数据转化为代码。
db_name = "wbh_2018"
excel_file = r"d:/iot2018/doc/设计/2018物博会展会表结构数据字典1.1.xlsx"
sheet_list = "交通性别统计,交通年龄统计,交通人数统计".split(",")
txt = excel_2_model(db_name,excel_file,sheet_list=sheet_list)
print(txt)
#with open(r"c:\Users\ken\Desktop\model.txt","w") as writer:
    #writer.write(txt)
    

class TrafficSexStat(models.Model):
    startdate = models.CharField(max_length=30,null=True)
    station_name = models.CharField(max_length=50,null=True)
    men_cnt = models.IntegerField(null=True)
    women_cnt = models.IntegerField(null=True)

    class Meta:
        db_table = 'traffic_sex_stat'

class TrafficAgeStat(models.Model):
    startdate = models.CharField(max_length=30,null=True)
    station_name = models.CharField(max_length=50,null=True)
    age_range = models.CharField(max_length=30,null=True)
    people_cnt = models.IntegerField(null=True)

    class Meta:
        db_table = 'traffic_age_stat'

class TrafficPeopleStat(models.Model):
    startdate = models.CharField(max_length=100,null=True)
    station_name = models.CharField(max_length=100,null=True)
    station_in_cnt = models.IntegerField(null=True)
    station_out_cnt = models.IntegerField(null=True)
    station_total_cnt = models.IntegerField(null=True)

    class Meta:
        db_table = 'traffic_people_stat'




In [9]:
def model(arr):
    hashTable = {}
    data = []
    for i in arr:
        try:
            hashTable[i]
        except NameError:
            var_exists = False
        else:
            hashTable[arr[i]] = True
            data.append(arr[i])
#         if hashTable[arr[i]]:
#             hashTable[arr[i]] = True
#             data.append(arr[i])
    return data
model([1,35,22,43,22,8,56,35,41])

KeyError: 1